In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
import statsmodels.api as sm
import json
import os
warnings.filterwarnings('ignore')
from datequarter import DateQuarter
from datetime import datetime

In [50]:
df_panjiva = pd.read_stata('panjiva.dta')

In [51]:
df_panjiva.shape

(1171617, 69)

In [52]:
df_panjiva[['gvkey_hs6', 'gvkey', 'hs6', 'date', 'tariffhs6', 'after']].head(10)

,gvkey_hs6,gvkey,hs6,date,tariffhs6,after
0,10000_060420,10000,060420,2017-01-01,0.0,0.0
1,10000_060420,10000,060420,2017-04-01,0.0,0.0
2,10000_060420,10000,060420,2017-07-01,0.0,0.0
3,10000_060420,10000,060420,2017-10-01,0.0,0.0
4,10000_060420,10000,060420,2018-01-01,0.0,0.0
5,10000_060420,10000,060420,2018-04-01,0.0,0.0
6,10000_060420,10000,060420,2018-07-01,0.0,1.0
7,10000_060420,10000,060420,2018-10-01,0.0,1.0
8,10000_060420,10000,060420,2019-01-01,0.0,1.0
9,10000_060420,10000,060420,2019-04-01,0.0,1.0


In [57]:
df_panjiva['hs6'].dtype

dtype('O')

In [53]:
div_list = ['vol_p', 'volchina', 'volindia', 'volvietnam', 'volmexico', 'volcanada', 'div_p', 'divchina']

In [54]:
hs2_list = list(set(df_panjiva['hs2']))

In [55]:
div = div_list[0]
# hs2 = hs2_list[0]

In [48]:
div

'vol_p'

In [38]:
df = pd.DataFrame()

# for div in div_list:
df_ = pd.DataFrame(df_panjiva[['hs2', div]].groupby('hs2')[div].mean())
df = pd.concat([df,df_], axis='columns')

df

,vol_p,volchina,volindia,volvietnam,volmexico,volcanada,div_p,divchina
hs2,,,,,,,,
00,100.000000,42.857143,14.285714,0.000000,0.000000,0.000000,NaN,0.000000
01,100.000000,33.333332,16.666666,0.000000,0.000000,11.111111,0.000000,0.000000
02,95.660912,15.901126,0.933551,0.589760,1.948729,16.126217,0.774732,0.015977
03,94.379082,32.301453,7.942838,8.344081,0.723676,3.407305,0.336659,0.046280
04,93.698662,13.238522,2.867800,1.464997,2.194932,3.863208,0.151418,0.001789
...,...,...,...,...,...,...,...,...
95,96.573318,76.870247,1.400039,1.241714,0.402680,0.356757,0.263592,0.085313
96,96.884811,71.828751,1.976258,1.018170,0.826494,0.361773,0.014774,0.060240
97,98.316826,40.243023,0.581395,0.357267,0.290698,2.979942,-0.280488,0.008721


In [ ]:
# TAB

In [117]:
df_plot = pd.read_stata('plot2.dta')

In [142]:
set(df_plot.year)

{2015.0, 2016.0, 2017.0, 2018.0, 2019.0}

In [144]:
df_plot = df_plot[df_plot['year']>=2017]

In [145]:
def get_plot1():

    df_plot1 = df_plot[['classification', 'tariffhs6', 'time', 'classif', 'vol', 'vol_p', 'volchina', 'volindnam', 'volmexda']]

    df_class = df_plot1.groupby(['classification', 'tariffhs6', 'time']).mean()
    df_class.reset_index(inplace=True)

    df_total = df_plot1.groupby(['tariffhs6', 'time']).mean()
    df_total.reset_index(inplace=True)
    df_total['classification'] = 0

    df = df_class.append(df_total)
    df['classification'] = df['classification'].astype(int)
    return df

In [146]:
df_plot1 = get_plot1()

In [147]:
df_plot1.head(4)

,classification,tariffhs6,time,vol,vol_p,volchina,volindnam,volmexda
0,1,0.0,after,14.879890,94.662849,10.756673,2.812948,13.853785
1,1,0.0,before,14.477104,95.032616,17.430786,1.291712,11.994617
2,1,1.0,after,12.564308,94.776939,26.033352,15.733186,7.637279
3,1,1.0,before,10.768200,94.364410,29.213793,14.304441,6.775394


In [148]:
def get_plot2():

    df_plot1 = df_plot[['classification', 'tariffhs6', 'time', 'D30', 'D40', 'D50', 'D60', 'D70', 'D80']]

    df_class = df_plot1.groupby(['classification', 'tariffhs6', 'time']).sum()
    df_class.reset_index(inplace=True)

    df_total = df_plot1.groupby(['tariffhs6', 'time']).sum()
    df_total.reset_index(inplace=True)
    df_total['classification'] = 0

    df = df_class.append(df_total)
    df['classification'] = df['classification'].astype(int)
    return df

In [149]:
df_plot2 = get_plot2()

In [150]:
df_plot2.head(4)

,classification,tariffhs6,time,D30,D40,D50,D60,D70,D80
0,1,0.0,after,0.0,10.0,41.0,69.0,97.0,122.0
1,1,0.0,before,1.0,10.0,34.0,52.0,78.0,107.0
2,1,1.0,after,1.0,7.0,68.0,92.0,163.0,217.0
3,1,1.0,before,3.0,19.0,82.0,122.0,178.0,219.0


In [151]:
df = pd.concat([df_plot1, df_plot2], keys=['classification', 'tariffhs6', 'time'], axis='columns')
df

classification                                                     \
   classification tariffhs6    time        vol      vol_p   volchina   
0               1       0.0   after  14.879890  94.662849  10.756673   
1               1       0.0  before  14.477104  95.032616  17.430786   
2               1       1.0   after  12.564308  94.776939  26.033352   
3               1       1.0  before  10.768200  94.364410  29.213793   
4               2       0.0   after   6.285631  96.501251  12.313176   
..            ...       ...     ...        ...        ...        ...   
59             15       1.0  before  21.652411  96.347900  67.555878   
0               0       0.0   after  16.767815  95.598389  56.973263   
1               0       0.0  before  14.598932  95.398033  57.870724   
2               0       1.0   after  13.829926  96.286507  49.177502   
3               0       1.0  before  13.192484  96.343491  50.098347   

                              tariffhs6                                  \
    volindnam   volmexda classification tariffhs6    time    D30    D40   
0    2.812948  13.853785              1       0.0   after    0.0   10.0   
1    1.291712  11.994617              1       0.0  before    1.0   10.0   
2   15.733186   7.637279              1       1.0   after    1.0    7.0   
3   14.304441   6.775394              1       1.0  before    3.0   19.0   
4    4.869764   7.305409              2       0.0   after    0.0    0.0   
..        ...        ...            ...       ...     ...    ...    ...   
59   4.724055   0.925233             15       1.0  before    4.0   59.0   
0    8.448165   1.801621              0       0.0   after   92.0  534.0   
1    7.481044   1.871497              0       0.0  before  153.0  721.0   
2    5.779146   2.994484              0       1.0   after  160.0  940.0   
3    5.219385   2.671770              0       1.0  before  131.0  946.0   

                                      
       D50     D60      D70      D80  
0     41.0    69.0     97.0    122.0  
1     34.0    52.0     78.0    107.0  
2     68.0    92.0    163.0    217.0  
3     82.0   122.0    178.0    219.0  
4      9.0    16.0     35.0     64.0  
..     ...     ...      ...      ...  
59   324.0   704.0   1181.0   1658.0  
0   1923.0  4154.0   6496.0   9008.0  
1   2246.0  4434.0   6860.0   9269.0  
2   3903.0  8357.0  13680.0  18932.0  
3   3776.0  8088.0  13319.0  18306.0  

[64 rows x 17 columns]

In [152]:
df.reset_index(inplace=True)
df

index classification                                                     \
         classification tariffhs6    time        vol      vol_p   volchina   
0      0              1       0.0   after  14.879890  94.662849  10.756673   
1      1              1       0.0  before  14.477104  95.032616  17.430786   
2      2              1       1.0   after  12.564308  94.776939  26.033352   
3      3              1       1.0  before  10.768200  94.364410  29.213793   
4      4              2       0.0   after   6.285631  96.501251  12.313176   
..   ...            ...       ...     ...        ...        ...        ...   
59    59             15       1.0  before  21.652411  96.347900  67.555878   
60     0              0       0.0   after  16.767815  95.598389  56.973263   
61     1              0       0.0  before  14.598932  95.398033  57.870724   
62     2              0       1.0   after  13.829926  96.286507  49.177502   
63     3              0       1.0  before  13.192484  96.343491  50.098347   

                              tariffhs6                                  \
    volindnam   volmexda classification tariffhs6    time    D30    D40   
0    2.812948  13.853785              1       0.0   after    0.0   10.0   
1    1.291712  11.994617              1       0.0  before    1.0   10.0   
2   15.733186   7.637279              1       1.0   after    1.0    7.0   
3   14.304441   6.775394              1       1.0  before    3.0   19.0   
4    4.869764   7.305409              2       0.0   after    0.0    0.0   
..        ...        ...            ...       ...     ...    ...    ...   
59   4.724055   0.925233             15       1.0  before    4.0   59.0   
60   8.448165   1.801621              0       0.0   after   92.0  534.0   
61   7.481044   1.871497              0       0.0  before  153.0  721.0   
62   5.779146   2.994484              0       1.0   after  160.0  940.0   
63   5.219385   2.671770              0       1.0  before  131.0  946.0   

                                      
       D50     D60      D70      D80  
0     41.0    69.0     97.0    122.0  
1     34.0    52.0     78.0    107.0  
2     68.0    92.0    163.0    217.0  
3     82.0   122.0    178.0    219.0  
4      9.0    16.0     35.0     64.0  
..     ...     ...      ...      ...  
59   324.0   704.0   1181.0   1658.0  
60  1923.0  4154.0   6496.0   9008.0  
61  2246.0  4434.0   6860.0   9269.0  
62  3903.0  8357.0  13680.0  18932.0  
63  3776.0  8088.0  13319.0  18306.0  

[64 rows x 18 columns]

In [153]:
df.columns = df.columns.droplevel(0)
df

,,classification,tariffhs6,time,vol,vol_p,volchina,volindnam,volmexda,classification,tariffhs6,time,D30,D40,D50,D60,D70,D80
0,0,1,0.0,after,14.879890,94.662849,10.756673,2.812948,13.853785,1,0.0,after,0.0,10.0,41.0,69.0,97.0,122.0
1,1,1,0.0,before,14.477104,95.032616,17.430786,1.291712,11.994617,1,0.0,before,1.0,10.0,34.0,52.0,78.0,107.0
2,2,1,1.0,after,12.564308,94.776939,26.033352,15.733186,7.637279,1,1.0,after,1.0,7.0,68.0,92.0,163.0,217.0
3,3,1,1.0,before,10.768200,94.364410,29.213793,14.304441,6.775394,1,1.0,before,3.0,19.0,82.0,122.0,178.0,219.0
4,4,2,0.0,after,6.285631,96.501251,12.313176,4.869764,7.305409,2,0.0,after,0.0,0.0,9.0,16.0,35.0,64.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,59,15,1.0,before,21.652411,96.347900,67.555878,4.724055,0.925233,15,1.0,before,4.0,59.0,324.0,704.0,1181.0,1658.0
60,0,0,0.0,after,16.767815,95.598389,56.973263,8.448165,1.801621,0,0.0,after,92.0,534.0,1923.0,4154.0,6496.0,9008.0
61,1,0,0.0,before,14.598932,95.398033,57.870724,7.481044,1.871497,0,0.0,before,153.0,721.0,2246.0,4434.0,6860.0,9269.0
62,2,0,1.0,after,13.829926,96.286507,49.177502,5.779146,2.994484,0,1.0,after,160.0,940.0,3903.0,8357.0,13680.0,18932.0


In [154]:
df = df.loc[:,~df.columns.duplicated()].copy()
df

,,classification,tariffhs6,time,vol,vol_p,volchina,volindnam,volmexda,D30,D40,D50,D60,D70,D80
0,0,1,0.0,after,14.879890,94.662849,10.756673,2.812948,13.853785,0.0,10.0,41.0,69.0,97.0,122.0
1,1,1,0.0,before,14.477104,95.032616,17.430786,1.291712,11.994617,1.0,10.0,34.0,52.0,78.0,107.0
2,2,1,1.0,after,12.564308,94.776939,26.033352,15.733186,7.637279,1.0,7.0,68.0,92.0,163.0,217.0
3,3,1,1.0,before,10.768200,94.364410,29.213793,14.304441,6.775394,3.0,19.0,82.0,122.0,178.0,219.0
4,4,2,0.0,after,6.285631,96.501251,12.313176,4.869764,7.305409,0.0,0.0,9.0,16.0,35.0,64.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,59,15,1.0,before,21.652411,96.347900,67.555878,4.724055,0.925233,4.0,59.0,324.0,704.0,1181.0,1658.0
60,0,0,0.0,after,16.767815,95.598389,56.973263,8.448165,1.801621,92.0,534.0,1923.0,4154.0,6496.0,9008.0
61,1,0,0.0,before,14.598932,95.398033,57.870724,7.481044,1.871497,153.0,721.0,2246.0,4434.0,6860.0,9269.0
62,2,0,1.0,after,13.829926,96.286507,49.177502,5.779146,2.994484,160.0,940.0,3903.0,8357.0,13680.0,18932.0


In [155]:
df.index

RangeIndex(start=0, stop=64, step=1)

In [156]:
df.to_csv(os.path.join('DATA', 'div_class.csv'))

In [10]:
df = pd.read_fwf("total_stats.txt") 
df.head()

,Unnamed: 0,Mean,SD,p25,p50,p75,N
0,dap,"2,033.33","18,265.69",37.24,54.48,79.26,"936,579"
1,roa,-0.89,948.49,0.32,1.18,2.15,"944,865"
2,roe,-24.48,"8,986.67",0.62,1.23,1.87,"930,323"
3,roi,3.03,223.06,0.74,1.91,3.42,"943,994"
4,mkvaltq,"43,486.54",101840.29,"1,781.02","7,174.33","31,774.37","845,275"


In [11]:
df.to_csv('total_stats.tsv', sep='&', index=False)

In [12]:
df = pd.read_fwf("after_stats.txt") 
df.to_csv('after_stats.tsv', sep='&', index=False)

In [13]:
df = pd.read_fwf("before_stats.txt") 
df.to_csv('before_stats.tsv', sep='&', index=False)